To do:

- Read demand data
- Mean demand --> discrete requests (random issue+pick-up time)
    - Is there any demand in this running instance for this od-pair?
    - If there is demand > 0 for an od, then put it at random points during the peak hour (=attach a pick-up time between 0 and 60)
- Import OD-matrix (travel times)

In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
#lambdapeak: Mean demand (arrivals/min.) to travel towards the city center during peak hours
#mupeak: Mean demand (arrival/min.) to travel towards the terminal during peak hours

lambdapeak = pd.read_excel('Data/Demand rates/DemandInputSmall3.xlsx',sheet_name=0,header=0, index_col=(0,1), dtype=float)
mupeak = pd.read_excel('Data/Demand rates/DemandInputSmall3.xlsx',sheet_name=1,header=0, index_col=(0,1), dtype=float)

In [9]:
# init parameters:

size = 'small' # incorportate these in the function!
time_of_day = 'peak'
scenario = 2

In [10]:
#TODO: test on the bigger instances!

def get_scenario_mean_demand(direction, scen): #add size as a parameter later
    if direction == 'city':
        df = lambdapeak
        n = len(df.iloc[0])
        a = 1

        return df.loc[(scen,1):(scen,n),1:n]    

    else: 
        df = mupeak 
        n = len(df.iloc[0])
        a = n
    
        return df.loc[(scen,a):(scen,2*n-1),1:2*n-1]    
    

In [11]:
get_scenario_mean_demand('city', scenario)
#get_scenario_mean_demand('terminal', scenario)

1    2    3
2 1  0.0  0.1  4.0
  2  0.0  0.0  4.0
  3  0.0  0.0  0.0

In [7]:
def convert_md_todict(df_meandemand_city, df_meandemand_terminal, scen):
    n = len(df_meandemand_city.iloc[0])
    demand_dict = {}
    
    for i in range(1,n+1):
        for j in range(1,n+1):
            demand_dict[(i,j)] = df_meandemand_city.loc[(scen,i),j]
            
    for i in range(n,2*n):
        for j in range(n,2*n):
            demand_dict[(i,j)] = df_meandemand_terminal.loc[(scen,i),j]
    
    return demand_dict
    

In [9]:
#calculate whether for this loading instance there are any requests per od-pair

small_meandemand = convert_md_todict(lambdapeak, mupeak, scenario)
small_meandemand

{(1, 1): 0.0,
 (1, 2): 0.1,
 (1, 3): 4.0,
 (2, 1): 0.0,
 (2, 2): 0.0,
 (2, 3): 4.0,
 (3, 1): 0.0,
 (3, 2): 0.0,
 (3, 3): 0.0,
 (3, 4): 0.15,
 (3, 5): 0.05,
 (4, 3): 0.0,
 (4, 4): 0.0,
 (4, 5): 0.0233333333333333,
 (5, 3): 0.0,
 (5, 4): 0.0,
 (5, 5): 0.0}

In [10]:
# this function generates the discrete requests 
# Q: I've made it now so that every OD-pair where there is a mean demand > 0 has at least 1 request
# I now have a lot of requests close to each other, at which point can I aggregate?

peak_hour_duration = 60

def generate_requests(mean_demand):

    requests = {}

    for od in mean_demand.keys():
        if mean_demand[od] > 0:
    
            requests_od = []
            requests[od] = requests_od
            requests[od].append(0)

            t=0

            while t < peak_hour_duration:
                delta_t = np.random.exponential(1/mean_demand[od], 1)[0]  #the time before the next passenger arrives is sampled from an exponential distr.                              
                t += delta_t
            
                if t > 60:
                    break
                else:
                    requests[od].append(t) #if no request yet is at that time

    return requests

small_requests = generate_requests(small_meandemand)
small_requests


{(1, 2): [0,
  28.407113207489235,
  32.17669168539131,
  40.236454807666185,
  42.17632695554921,
  50.50785170520504,
  55.70797952881701,
  58.66157588784098],
 (1, 3): [0,
  0.1403039226465128,
  0.20978743783730414,
  0.21631632923156086,
  0.4281202641242935,
  0.5071587541090863,
  1.0027513055631039,
  1.0863795908753289,
  1.1665300400841745,
  1.3783385157453578,
  1.6856287815563562,
  1.9200114705257139,
  2.129864580396628,
  2.1916648507755148,
  2.3473259136371643,
  2.354725611587812,
  2.410616036755492,
  2.5607007024000072,
  2.6223601718351692,
  2.824815250699431,
  3.0141667409527573,
  3.0744490680552103,
  3.266362590603292,
  4.1817065240613065,
  4.5109544924087395,
  4.621483818658806,
  5.31707927800999,
  5.503083265658093,
  5.533734367763384,
  5.920013815727957,
  6.024536998062504,
  6.4630110335647535,
  6.480678561398577,
  6.712578932505324,
  6.7476352119406,
  6.917977615350637,
  7.037249709676599,
  7.0516514293575465,
  7.1070842153398,
  7.5071

In [15]:
# create individual requests. R = ((o,d), pickuptime)

lst_small_requests = []

for k in small_requests.keys():
    for v in small_requests[k]:
        lst_small_requests.append((k, v, 0))
        
lst_small_requests
        

[((1, 2), 0, 0),
 ((1, 2), 28.407113207489235, 0),
 ((1, 2), 32.17669168539131, 0),
 ((1, 2), 40.236454807666185, 0),
 ((1, 2), 42.17632695554921, 0),
 ((1, 2), 50.50785170520504, 0),
 ((1, 2), 55.70797952881701, 0),
 ((1, 2), 58.66157588784098, 0),
 ((1, 3), 0, 0),
 ((1, 3), 0.1403039226465128, 0),
 ((1, 3), 0.20978743783730414, 0),
 ((1, 3), 0.21631632923156086, 0),
 ((1, 3), 0.4281202641242935, 0),
 ((1, 3), 0.5071587541090863, 0),
 ((1, 3), 1.0027513055631039, 0),
 ((1, 3), 1.0863795908753289, 0),
 ((1, 3), 1.1665300400841745, 0),
 ((1, 3), 1.3783385157453578, 0),
 ((1, 3), 1.6856287815563562, 0),
 ((1, 3), 1.9200114705257139, 0),
 ((1, 3), 2.129864580396628, 0),
 ((1, 3), 2.1916648507755148, 0),
 ((1, 3), 2.3473259136371643, 0),
 ((1, 3), 2.354725611587812, 0),
 ((1, 3), 2.410616036755492, 0),
 ((1, 3), 2.5607007024000072, 0),
 ((1, 3), 2.6223601718351692, 0),
 ((1, 3), 2.824815250699431, 0),
 ((1, 3), 3.0141667409527573, 0),
 ((1, 3), 3.0744490680552103, 0),
 ((1, 3), 3.266362590

In [16]:
requests = open("Data/requests.pkl", "wb")
pickle.dump(lst_small_requests, requests)
requests.close()